## Import necessary libraries

In [0]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
##from pattern import en
from scipy import spatial
import pickle

In [0]:
##!pip install pattern

## Global Variables

In [0]:
data_repository = 'candidates_2019_05_24_16_24.csv'
##job_repository = 'indeed_job_dataset_1.csv'
##word2vecModel = 'resume_word2vec'
##word2vecResume = 'resume_w2v_array'

In [3]:
#Mount drive
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
project_path ='/content/drive/My Drive/Capstone/'

In [5]:
pwd

'/content'

In [0]:
import os

In [0]:
os.chdir(project_path)


In [8]:
pwd

'/content/drive/My Drive/Capstone'

## Functions

In [0]:
def get_resume_data():
    dfdata = pd.read_csv(data_repository)
    dfdata['whole_text'] = pd.Series(dfdata.fillna('').values.tolist()).str.join(' ')
    return dfdata

In [0]:
def get_jobprofile_data():
    dfjob = pd.read_csv(job_repository)
    #dfjob['whole_text'] = pd.Series(dfjob.fillna('').values.tolist()).str.join(' ')
    return dfjob

In [0]:
def create_w2v_model(dataset):    
    alltext = ' '  
    for index, row in dataset.iterrows():
        alltext = alltext + " " + row['whole_text']   
    alltext = alltext.lower()
    vector = []
    for sentence in en.parsetree(alltext, tokenize=True, lemmata=True, tags=True):
        temp = []
        for chunk in sentence.chunks:
            for word in chunk.words:
                if word.tag == 'NN' or word.tag == 'VB':
                    temp.append(word.lemma)
        vector.append(temp)
    model = Word2Vec(vector, size=200, window=5, min_count=3, workers=4)
    model.save(word2vecModel)

In [0]:
def get_job_desc(skill, description):
    vector = []
    for sentence in en.parsetree(description, tokenize=True, lemmata=True, tags=True):
        for chunk in sentence.chunks:
            for word in chunk.words:
                if word.tag == 'NN' or word.tag == 'VB':
                    vector.append(word.lemma)
    for sentence in en.parsetree(skill, tokenize=True, lemmata=True, tags=True):
        for chunk in sentence.chunks:
            for word in chunk.words:
                vector.append(word.lemma)
    return vector

In [0]:
def create_w2v_resume(df_resume):
    D_w2v = []
    for index, row in df_resume.iterrows():
        print("Processing resume " + str(index))
        yd = row['whole_text']
        w2v = []
        for sentence in en.parsetree(yd.lower(), tokenize=True, lemmata=True, tags=True):
            for chunk in sentence.chunks:
                for word in chunk.words:
                    if word.lemma in model.wv.vocab:
                        w2v.append(model.wv[word.lemma])
                    else:
                        if word.lemma.lower() in model.wv.vocab:
                            w2v.append(model.wv[word.lemma.lower()])
        D_w2v.append((np.mean(w2v, axis=0),index))
    with open(word2vecResume, 'wb') as fp:
        pickle.dump(D_w2v, fp)

In [0]:
def get_recommend_initialrun(job_profile, df_resume):  
    #data = request.args.get('value')
    w2v = []
    job_profile = job_profile.lower()
    model = Word2Vec.load(word2vecModel)
    for sentence in en.parsetree(job_profile, tokenize=True, lemmata=True, tags=True):
        for chunk in sentence.chunks:
            for word in chunk.words:
                if word.lemma in model.wv.vocab:
                    w2v.append(model.wv[word.lemma])
                else:
                    if word.lemma.lower() in model.wv.vocab:
                        w2v.append(model.wv[word.lemma.lower()])
    Q_w2v = np.mean(w2v, axis=0)
    
    print("completed job profile screening")
    
    # Example of document represented by average of each document term vectors.
    D_w2v = []
    for index, row in df_resume.iterrows():
        print("Processing resume " + str(index))
        yd = row['whole_text']
        w2v = []
        for sentence in en.parsetree(yd.lower(), tokenize=True, lemmata=True, tags=True):
            for chunk in sentence.chunks:
                for word in chunk.words:
                    if word.lemma in model.wv.vocab:
                        w2v.append(model.wv[word.lemma])
                    else:
                        if word.lemma.lower() in model.wv.vocab:
                            w2v.append(model.wv[word.lemma.lower()])
        D_w2v.append((np.mean(w2v, axis=0),index))
    with open(word2vecResume, 'wb') as fp:
        pickle.dump(D_w2v, fp)
    
    # Make the retrieval using cosine similarity between query and document vectors.
    retrieval = []
    for i in range(len(D_w2v)):
        print('Calulating cosine similarity for resume: ' + str(i))
        retrieval.append((1 - spatial.distance.cosine(Q_w2v, D_w2v[i][0]),D_w2v[i][1]))
    retrieval.sort(reverse=True)
    return retrieval
    #with app.app_context(), app.test_request_context():
        #ret_data = {"cv1":url_for('static', filename="test/"+retrieval[0][1][retrieval[0][1].rfind('/')+1:]), "score1": str(round(retrieval[0][0], 4)), "cv2":url_for('static', filename="test/"+retrieval[1][1][retrieval[1][1].rfind('/')+1:]), "score2": str(round(retrieval[1][0], 4)),"cv3":url_for('static', filename="test/"+retrieval[2][1][retrieval[2][1].rfind('/')+1:]), "score3": str(round(retrieval[2][0], 4))   }
        #return jsonify(ret_data)
        #return ret_data

In [0]:
def get_recommend(job_profile, df_resume):  
    #data = request.args.get('value')
    w2v = []
    job_profile = job_profile.lower()
    model = Word2Vec.load(word2vecModel)
    for sentence in en.parsetree(job_profile, tokenize=True, lemmata=True, tags=True):
        for chunk in sentence.chunks:
            for word in chunk.words:
                if word.lemma in model.wv.vocab:
                    w2v.append(model.wv[word.lemma])
                else:
                    if word.lemma.lower() in model.wv.vocab:
                        w2v.append(model.wv[word.lemma.lower()])
    Q_w2v = np.mean(w2v, axis=0)
    
    # Document represented by average of each document term vectors.
    #D_w2v = []
    with open(word2vecResume, 'rb') as fp:
        D_w2v = pickle.load(fp)
    
    # Make the retrieval using cosine similarity between query and document vectors.
    retrieval = []
    for i in range(len(D_w2v)):
        retrieval.append((1 - spatial.distance.cosine(Q_w2v, D_w2v[i][0]),D_w2v[i][1]))
    retrieval.sort(reverse=True)
    return retrieval
    #with app.app_context(), app.test_request_context():
        #ret_data = {"cv1":url_for('static', filename="test/"+retrieval[0][1][retrieval[0][1].rfind('/')+1:]), "score1": str(round(retrieval[0][0], 4)), "cv2":url_for('static', filename="test/"+retrieval[1][1][retrieval[1][1].rfind('/')+1:]), "score2": str(round(retrieval[1][0], 4)),"cv3":url_for('static', filename="test/"+retrieval[2][1][retrieval[2][1].rfind('/')+1:]), "score3": str(round(retrieval[2][0], 4))   }
        #return jsonify(ret_data)
        #return ret_data

## Implementation

In [0]:
dfdata = get_resume_data()

In [11]:
dfdata.head(2)

,First Name,Last Name,Position,Company,Location,Experience1,Experience2,Experience3,Experience4,Experience5,Experience6,Experience7,Experience8,Experience9,Experience10,Experience11,Experience12,Experience13,Experience14,Experience15,Experience16,Experience17,Experience18,Experience19,Experience20,Experience21,Experience22,Experience23,Experience24,Experience25,Experience26,Experience27,Experience28,Experience29,Experience30,Experience31,Experience32,Experience33,Experience34,Experience35,Experience36,Experience37,Skill,Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Education11,Education12,Education13,Education14,whole_text
0,Igor,Beliaev,"Senior Manager, Data Science",Gogo,"Chicago, IL, United States","Lead Data Analyst, Gogo, 2015-2016\nMigrated m...","Senior Data Analyst, Gogo, 2013-2015\nExtended...","Data Analyst, Gogo, 2011-2013\nData analysis o...","Data Analyst, Market Dynamics, Inc, 2006-2011\...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,".NET, Access, Agile Methodologies, Algorithms,...","MS, Mathematics, Computer Science, The Univers...","Assoc., Management, Banking and Finance, Briti...","BSc., Applied Mathematics, Moscow Aviation Ins...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Igor Beliaev Senior Manager, Data Science Gogo..."
1,Adam,Owens,"Data Scientist, Revenue","Snapchat, Inc.","Los Angeles, CA, United States","Marketing Data Analyst, Snapchat, Inc., 2017-2...","Data Analytics Instructor, Product School, 201...","Data Strategy Consultant / Founder, Vault Econ...","Data Scientist, Intern, WeWork, 2017-2017\nBui...","Data Science, WeWork, 2017-","Technical Consultant - Marketing Analytics, Pr...","Investment Banking, Wells Fargo, 2010-2014\nCo...","Corporate Financial Analyst, Citigroup, 2009-2...","Economics Research Assistant, Brigham Young Un...","Research Assistant Intern, Center for Strategi...","Business Strategy, Crewspark, 2015-2015\n- Lea...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Accounting, Apache Spark, Bloomberg, Business ...","Certificate / Masters Degree, Computer Science...","BA, Economics, Brigham Young University, 2005-...","Data Science Bootcamp, NYC Data Science Academ...","Classwork, Computer Science, The Johns Hopkins...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Adam Owens Data Scientist, Revenue Snapchat, I..."


In [0]:
##dfjob = get_jobprofile_data()

In [0]:
##create_w2v_model(dfdata)

In [0]:
#model = Word2Vec.load("resume_word2vec")

In [0]:
#if 'matlab' in word_vectors.vocab:
 #   print("Yes")
#else:
 #   print("No")

Yes


In [0]:
##job_profile = dfjob.Skill[0] + dfjob.Description[0]

In [0]:
##result = get_recommend(job_profile, dfdata)

In [12]:
dfdata['whole_text'][0]

'Igor Beliaev Senior Manager, Data Science Gogo Chicago, IL, United States Lead Data Analyst, Gogo, 2015-2016\nMigrated machine data processing solution to Hadoop on-prem cluster, lead team of junior developers in migration of legacy applications to big data platform, building new analytical capabilities (Hadoop, hive, spark, pig, java, python) Senior Data Analyst, Gogo, 2013-2015\nExtended scope of corporate data warehouse with operational data and built analytics and applications to support fleet maintenance and operations groups (SQL Server, SSIS, Tableau, .Net) Data Analyst, Gogo, 2011-2013\nData analysis of airborne and ground based hardware to support operations groups troubleshooting and increase efficiency (.Net, SQL Server, MySQL, Tableau) Data Analyst, Market Dynamics, Inc, 2006-2011\nCoordinate data research and full cycle software development between multi-cultural, geographically distributed programmers groups to support data analysis, data mining, processing and analysis 

In [14]:
len(dfdata)

1000

In [0]:
dfdata1 = dfdata.head(20)

In [70]:
len(dfdata1)

20

In [0]:
import tensorflow as tf
import tensorflow_hub as hub

In [0]:
url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

In [0]:
def elmo_vectors(x):
  embeddings = embed(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [0]:
elmo_train = [elmo_vectors(dfdata1['whole_text'])]

In [73]:
type(elmo_train)

list

In [0]:
elmo_train = np.asarray(elmo_train)

In [75]:
type(elmo_train)

numpy.ndarray

In [76]:
elmo_train

array([[[-0.09034549,  0.01817906,  0.03737507, ...,  0.03317315,
         -0.00984211, -0.11262437],
        [-0.13330315, -0.04385478,  0.10310254, ...,  0.03069372,
          0.09470636, -0.12958546],
        [-0.08792215,  0.1135233 , -0.00591619, ...,  0.00296325,
         -0.0104468 , -0.2170866 ],
        ...,
        [-0.1016537 , -0.03657316,  0.06252412, ..., -0.00271141,
         -0.01541165, -0.1655155 ],
        [-0.1415498 ,  0.08503515, -0.0303641 , ..., -0.01843785,
          0.0048802 , -0.19220376],
        [-0.1348576 ,  0.03137769,  0.02285027, ...,  0.01124741,
         -0.08620825, -0.14283521]]], dtype=float32)

In [77]:
elmo_train.shape

(1, 20, 1024)

In [0]:
nsamples, nx, ny = elmo_train.shape


In [79]:
print(nsamples)
print(nx)
print(ny)

1
20
1024


In [0]:
elmo_train = elmo_train.reshape((nx,nsamples*ny))

In [81]:
elmo_train.shape

(20, 1024)

In [0]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
y = pca.fit_transform(elmo_train)
## model1=pca.transform(np.array(list(model1.values())))
from sklearn.manifold import TSNE

y = TSNE(n_components=3).fit_transform(y)

In [85]:
y.shape

(20, 3)

In [0]:
sentences = dfdata1['whole_text']

In [0]:
from IPython.display import HTML

In [91]:
##@title Sementic search
##@markdown Enter a set of words to find matching sentences. 'results_returned' can beused to modify the number of matching sentences retured. To view the code behind this cell, use the menu in the top right to unhide...

search_string = "Manager,Python" #@param {type:"string"}
results_returned = "3" #@param [1, 2, 3]

from sklearn.metrics.pairwise import cosine_similarity


embeddings2 = embed(
    [search_string],
    signature="default",
    as_dict=True)["default"]

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  sess.run(tf.tables_initializer())
  search_vect = sess.run(embeddings2)
  

cosine_similarities = pd.Series(cosine_similarity(search_vect, elmo_train).flatten())
output =""
for i,j in cosine_similarities.nlargest(int(results_returned)).iteritems():
  output +='<p style="font-family:verdana; font-size:110%;"> '
  for i in sentences[i].split():
    if i.lower() in search_string:
      output += " <b>"+str(i)+"</b>"
    else:
      output += " "+str(i)
  output += "</p><hr>"
    
output = '<h3>Results:</h3>'+output
display(HTML(output))
#   print(sentences[i])
#   print('\n')
